In [3]:
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn.functional as F
from pudb import set_trace;
import time

with open('data/countries_s1') as f:
    facts = f.read().splitlines()
facts = [el.split(',') for el in facts]
preds = [fact[0] for fact in facts]
subjs = [fact[1] for fact in facts]
objs = [fact[2] for fact in facts]

unique = sorted(list(set(preds)))
num_unique = len(unique)
num_predicates = num_unique
predsToIdx = dict(zip(unique,range(num_unique)))
idxToPreds = dict(zip(range(num_unique),unique))
print(idxToPreds)

unique = sorted(list(set(subjs+objs)))
num_unique = len(unique)
num_constants = num_unique
consToIdx = dict(zip(unique,range(num_unique)))
idxToCons = dict(zip(range(num_unique),unique))

facts = np.array([(predsToIdx[preds[i]],consToIdx[subjs[i]], consToIdx[objs[i]]) for i in range(len(facts))])
data = np.zeros((num_predicates,num_constants,num_constants))

#data: idx0->predicate, idx1->subj, idx2->obj
data[facts[:,0],facts[:,1],facts[:,2]] = 1
no_facts = int(np.sum(data))
data = torch.from_numpy(data)

predicates = torch.eye(num_predicates)
constants = torch.eye(num_constants)

knowledge_pos = (data==1).nonzero()
data_aux = knowledge_pos
knowledge_pos = torch.cat((predicates[knowledge_pos[:,0]],
                           constants[knowledge_pos[:,1]],
                           constants[knowledge_pos[:,2]]),dim=1)

num_feats_per_fact = knowledge_pos.size()[1]

#reading the test set
with open('data/s1_test') as f:
    test = f.read().splitlines()
test = [el.split(',') for el in test]
preds = [fact[0] for fact in test]
subjs = [fact[1] for fact in test]
objs = [fact[2] for fact in test]

test = np.array([(predsToIdx[preds[i]],consToIdx[subjs[i]], consToIdx[objs[i]]) for i in range(len(test))])
ts_data = np.zeros((num_predicates,num_constants,num_constants))
ts_data[test[:,0],test[:,1],test[:,2]] = 1
ts_data = torch.from_numpy(ts_data)
test = (ts_data==1).nonzero()
test = torch.cat((predicates[test[:,0]],
                   constants[test[:,1]],
                   constants[test[:,2]]),dim=1)

#sample num_samples as a connected subgraph of the input data
#   basically, perform num_samples steps, each adding one more fact
#   that is connected to at least one constant in the sample
#data: a tensor of the form num_preds,num_cons,num_cons where a 1
#   means that the fact composed of pred,cons1,cons2 is true
#num_samples: number of samples to be gotten
#returns sample: a tensor of the form num_samples*num_feats_per_fact
def sample_neighbors(num_samples,data):
    data_source_tmp = data.clone()
    data_tmp = torch.zeros_like(data)
    sample = torch.zeros(0,num_feats_per_fact,dtype=torch.long)
    #choose one random constant
    idx = torch.randperm(num_constants)[0].unsqueeze(0)
    for _ in range(num_samples):
    #     print('data_source',data_source_tmp)
        #subset your possible choices to where idx is subject or object
        data_tmp[:,idx,:] = data_source_tmp[:,idx,:]
        data_tmp[:,:,idx] = data_source_tmp[:,:,idx]
        #choose one at random
        new_fact = data_tmp.nonzero()
        if new_fact.size()[0] == 0:
            break
        chosen = torch.randperm(new_fact.size()[0])[0]
        new_fact = new_fact[chosen,:].unsqueeze(0)
        #add fact to sample
        sample = torch.cat((sample,new_fact),dim=0)
        #set chosen fact to zero (avoiding choosing it again)
        data_source_tmp[new_fact[:,0],new_fact[:,1],new_fact[:,2]] = 0
        #add new idx in the fact
        idx = torch.cat((idx,new_fact[:,1],new_fact[:,2]))
        idx = torch.unique(idx)
    sample = torch.cat((predicates[sample[:,0]],
                        constants[sample[:,1]],
                        constants[sample[:,2]]),dim=1)
    return sample




#helps removing repeated predicted facts -> same embeddings and constants, probably different scores
#this is of complexity K^3, could be optimized
def leaveTopK(preds,K):
    _,idx = torch.sort(preds[:,-1],descending=True)
    preds = preds[idx,:]
    out = preds[0,:].unsqueeze(0)
    for i in range(1,min(K,preds.size()[0])):
        t = preds[i,:].unsqueeze(0)
        if t[:,-1] == 0:
            break
        m,_ = torch.max(F.cosine_similarity(t[:,:-1].repeat(out.size()[0],1),out[:,:-1],dim=1),dim=0)
        if m<1:
            out = torch.cat((out,t),dim=0)
    return out

####FORWARD CHAINING
#input: some facts that can either be ground or predicted in previous steps
#output: predicted facts in this specific step (outer loop must gather all of them)
#computes the predictions of applying each rule to the facts, giving a score to each of them.
#leaves only topK scoring fact for each applied rule (not for the whole thing)
def forward_step(facts,K):
    num_facts = facts.size()[0]
    #rule 1
    # b1(x,y)<-b2(y,x)
    start_time = time.time()
    rule_expanded = rules[0].repeat(num_facts,1)
    preds_r1 = F.cosine_similarity(rule_expanded[:,num_predicates:],facts[:,:num_predicates],dim=1)
    preds_r1 = preds_r1*facts[:,-1]
    preds_r1 = preds_r1.unsqueeze(1)
    preds_r1 = torch.cat((rule_expanded[:,:num_predicates],
                         facts[:,num_predicates+num_constants:-1],
                         facts[:,num_predicates:num_predicates+num_constants],
                         preds_r1),dim=1)
    # print(preds_r1)
    preds_r1 = leaveTopK(preds_r1,K)
    
    #rule 2
    #b1(x,y)<-b2(x,z),b3(z,y)
    body1 = facts.repeat((1,num_facts)).view(-1,num_feats_per_fact+1)
    body2 = facts.repeat((num_facts,1))
    rule_expanded = rules[1].repeat(body1.size()[0],1)
    #previous scores
    preds_r2 = body1[:,-1]*body2[:,-1]
    #similarity between shared constants
    preds_r2 = preds_r2*F.cosine_similarity(body1[:,num_predicates+num_constants:-1],
                                            body2[:,num_predicates:num_predicates+num_constants],dim=1)
    #remove 0 scores to improve speed - if constant isn't shared we don't compute anything else
    non_zero = preds_r2.nonzero().squeeze()
    preds_r2 = preds_r2[non_zero]
    rule_expanded = rule_expanded[non_zero,:]
    body1 = body1[non_zero,:]
    body2 = body2[non_zero,:]
    
    #predicate of body1 with predicate of rule
    preds_r2 = preds_r2*F.cosine_similarity(rule_expanded[:,num_predicates:2*num_predicates],body1[:,:num_predicates],dim=1)
    #predicate of body2 with predicate of rule
    preds_r2 = preds_r2*F.cosine_similarity(rule_expanded[:,2*num_predicates:],body2[:,:num_predicates],dim=1)
    
    preds_r2 = preds_r2.unsqueeze(1)
    preds_r2 = torch.cat((rule_expanded[:,:num_predicates]
                         ,body1[:,num_predicates:num_predicates+num_constants]
                         ,body2[:,num_predicates+num_constants:-1]
                         ,preds_r2)
                        ,dim=1)
    #removing repeated facts and leaving ones with highest score
    preds_r2 = leaveTopK(preds_r2,K)
    out = torch.cat((preds_r1,preds_r2),dim=0)
    print("fws took %s" % (time.time() - start_time))
    return out
    # return preds_r2

    
####TRAINING
#dbg -> cherrypicked
# core_rel = Variable(knowledge_pos[[0,1,3]])
# target = Variable(knowledge_pos[2]).unsqueeze(0)
# target = Variable(knowledge_pos[[2,4],:])
#####sampling
target = Variable(knowledge_pos)
no_samples = 50

num_iters = 100
learning_rate = .1
lamb = 1

steps = 2
num_rules = 2
epsilon=.001

K = 30 ##For top K
#Find maximum similarity for each consequence in the set of facts contained in target
#if testing is true it finds the consequence with maximum similarity for each target
#if testing is true, returns the truth value of the matched predicted consequence for each target
#Inputs: 
#consequences: facts to be looked for maximum similarities across target
#target: set of facts that are assumed to be true
#testing: returns the probabilities of matched facts, a prediction is considered true if p>0.5
def find_max_similarities(consequences,target,testing=False):
    start_time = time.time()    
    num_consequences = consequences.size()[0]
    num_targets = target.size()[0]

    #each consequence repeated by the number of targets
    if testing:
        #for each target find max similarity across consequences
        tmp_c = consequences.repeat(num_targets,1)
        tmp_t = target.repeat(1,num_consequences).view(-1,num_feats_per_fact)
    else:
        #for each consequence compute the similarity with all targets
        tmp_c = consequences.repeat(1,num_targets).view(-1,num_feats_per_fact+1)
        tmp_t = target.repeat(num_consequences,1)

    sim = F.cosine_similarity(tmp_c[:,num_predicates:num_predicates+num_constants],
                              tmp_t[:,num_predicates:num_predicates+num_constants],dim=1)
    #only compute for non-zero values to speed up
    non_zero = sim.nonzero().squeeze()

    sim[non_zero] = sim[non_zero] * F.cosine_similarity(tmp_c[non_zero,num_predicates+num_constants:-1],
                                                        tmp_t[non_zero,num_predicates+num_constants:],dim=1)

    non_zero = sim.nonzero().squeeze()

    sim[non_zero] = sim[non_zero] * F.cosine_similarity(tmp_c[non_zero,:num_predicates]
                                                       ,tmp_t[non_zero,:num_predicates],dim=1)
    
    #for each consequence/target, get the maximum simlarity with the set of targets/consequences
    if testing:
        sim = sim.view(-1,num_consequences)
    else:
        sim = sim.view(-1,num_targets)
    m, idx = torch.max(sim,dim=1)
    print("fms took %s" % (time.time() - start_time))
    if testing:
        return m, tmp_c[idx,-1]
    return m

{0: 'locatedIn', 1: 'neighborOf'}


In [5]:
K_tmp = K
#rules should be:
#r1(x,y) <- r2(y,x)
#r1(x,y) <- r2(x,z),r3(z,x)
rules = [Variable(torch.Tensor([ 0.1000,  0.9000,  0.1000,  0.9000]), requires_grad=True),
         Variable(torch.Tensor([ 0.1000,  0.9000,  0.9000,  0.1000,  0.1000,  0.9000]), requires_grad=True)]
# rules = [Variable(torch.rand(num_predicates), requires_grad=True),
#          Variable(torch.Tensor([1, 1]), requires_grad=True)]
optimizer = torch.optim.Adam([
        {'params': rules}], 
        lr = learning_rate)

criterion = torch.nn.MSELoss(size_average=False)

rules_tmp = [torch.zeros_like(rule) for rule in rules]
for epoch in range(num_iters):
    for par in optimizer.param_groups:
        par['params'][1].data.clamp_(min=0.1,max=0.9)
        par['params'][0].data.clamp_(min=0.1,max=0.9)
    # ##sampling
    core_rel = torch.randperm(no_facts)
    # # target = core_rel[no_samples:]
    core_rel = core_rel[:no_samples]

    # core_rel = Variable(knowledge_pos[core_rel])
    core_rel = sample_neighbors(no_samples,data)
    # target = Variable(knowledge_pos)
    optimizer.zero_grad()
    facts = torch.cat((core_rel, Variable(torch.ones(core_rel.size()[0], 1))), 1)
    #will accumulate predictions separately to compare with target facts
    consequences = forward_step(facts,K_tmp)
    for step in range(1,steps):
        tmp = torch.cat((consequences,facts),dim=0)
        tmp = forward_step(tmp,K_tmp)
        consequences = torch.cat((consequences,tmp),dim=0)
    #LOSS
    loss = 0
    m = find_max_similarities(consequences,target)
    loss = torch.sum(lamb*consequences[:,-1]*(1 - consequences[:,-1]*m))
    print(epoch, 'losssssssssssssssssssss',loss.data[0])
    # print(sum([torch.sum(rules_tmp[i]-rules[i]) for i in range(num_rules)]))
    if loss < 10**-6 or sum([torch.sum(torch.abs(rules_tmp[i]-rules[i])) for i in range(num_rules)])<10**-5:
        break
    rules_tmp = [r.clone() for r in rules]
    loss.backward()
    optimizer.step()
    print(rules)


#computing test results



fws took 0.02703690528869629
fws took 0.08900976181030273
fms took 0.760657787322998
0 losssssssssssssssssssss tensor(10.3896)


/Users/aldo.pareja/miniconda3/envs/torch/lib/python3.6/site-packages/ipykernel_launcher.py:40: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


[tensor([ 7.4506e-09,  1.0000e+00,  1.4901e-08,  1.0000e+00]), tensor([ 2.0000e-01,  8.0002e-01,  1.0000e+00,  7.4506e-09,  1.4901e-08,
         1.0000e+00])]
fws took 0.018731117248535156
fws took 0.04936504364013672
fms took 0.8886420726776123
1 losssssssssssssssssssss tensor(11.4990)
[tensor([ 0.0001,  0.9999,  0.0060,  0.9940]), tensor([ 1.4346e-01,  8.6683e-01,  1.0000e+00, -6.9886e-06,  3.1210e-02,
         9.6879e-01])]
fws took 0.02013707160949707
fws took 0.05693793296813965
fms took 0.7201879024505615
2 losssssssssssssssssssss tensor(9.7532)
[tensor([ 0.0001,  0.9999,  0.0042,  0.9958]), tensor([ 0.0752,  0.9368,  0.9994,  0.0006,  0.0432,  0.9568])]
fws took 0.021111249923706055
fws took 0.04829525947570801
fms took 0.9696073532104492
3 losssssssssssssssssssss tensor(10.4479)
[tensor([-3.6202e-05,  1.0000e+00,  2.6676e-03,  9.9733e-01]), tensor([ 0.0754,  0.9423,  0.9995,  0.0005,  0.0275,  0.9725])]
fws took 0.02122783660888672
fws took 0.06456804275512695
fms took 0.791405

RuntimeError: dimension out of range (expected to be in range of [-1, 0], but got 1)

tensor([    76,    229,    540,   1108,   1182,   1316,   1604,   1687,
          1776,   2449,   2450,   2735,   2761,   3412,   4049,   4154,
          4157,   4158,   4159,   4161,   4362,   4461,   4975,   5609,
          5711,   6030,   6238,   6403,   6412,   6421,   6429,   6436,
          6437,   6484,   6830,   7247,   7702,   7786,   7790,   7872,
          7912,   8200,   8304,   8364,   8401,   8404,   8657,   8661,
          8666,   8668,   8745,   9307,   9516,  10099,  10538,  10987,
         12003,  12011,  12097,  12283,  12582,  12604,  12648,  13088,
         13095,  13367,  13706,  13733,  14392,  14595,  14596,  14604,
         14605,  14606,  14607,  14608,  14623,  14745,  15524,  15538,
         15759,  15760,  15761,  15762,  15763,  15764,  15826,  16283,
         16293,  16307,  16356,  16369,  16456,  16940,  16963,  17001,
         17661,  17671,  17974,  17982,  18158,  18612,  18732,  18776,
         18795,  18803,  19081,  19097,  19106,  19336,  19342, 

RuntimeError: dimension out of range (expected to be in range of [-1, 0], but got 1)

In [50]:
def find_max_similarities(consequences,target,testing=False):
    start_time = time.time()    
    num_consequences = consequences.size()[0]
    num_targets = target.size()[0]

    #each consequence repeated by the number of targets
    if testing:
        #for each target find max similarity across consequences
        tmp_c = consequences.repeat(num_targets,1)
        tmp_t = target.repeat(1,num_consequences).view(-1,num_feats_per_fact)
    else:
        #for each consequence compute the similarity with all targets
        tmp_c = consequences.repeat(1,num_targets).view(-1,num_feats_per_fact+1)
        tmp_t = target.repeat(num_consequences,1)

    sim = F.cosine_similarity(tmp_c[:,num_predicates:num_predicates+num_constants],
                              tmp_t[:,num_predicates:num_predicates+num_constants],dim=1)
    #only compute for non-zero values to speed up
    non_zero = sim.nonzero().squeeze()
    if non_zero.size()[0]==0:
        sim = torch.zeros_like(sim)
    else:
        sim[non_zero] = sim[non_zero] * F.cosine_similarity(tmp_c[non_zero,num_predicates+num_constants:-1],tmp_t[non_zero,num_predicates+num_constants:],dim=1)

    non_zero = sim.nonzero().squeeze()
    if non_zero.size()[0]==0:
        sim = torch.zeros_like(sim)
    else:
        sim[non_zero] = sim[non_zero] * F.cosine_similarity(tmp_c[non_zero,:num_predicates] ,tmp_t[non_zero,:num_predicates],dim=1)
    
    #for each consequence/target, get the maximum simlarity with the set of targets/consequences
    if testing:
        sim = sim.view(-1,num_consequences)
    else:
        sim = sim.view(-1,num_targets)
    m, idx = torch.max(sim,dim=1)
    print("fms took %s" % (time.time() - start_time))
    if testing:
        return m, tmp_c[idx,-1]
    return m


In [51]:
# print('computing test results')
# K_tmp = 400
# facts = torch.cat((knowledge_pos, Variable(torch.ones(knowledge_pos.size()[0], 1))), 1)
# consequences = forward_step(facts,K_tmp)
# for step in range(1,steps):
#     tmp = torch.cat((consequences,facts),dim=0)
#     tmp = forward_step(tmp,K_tmp)
#     consequences = torch.cat((consequences,tmp),dim=0)
m,p = find_max_similarities(consequences,test,testing=True)
true_positives = m[p>0.5]
true_positives = (true_positives>0.5).nonzero()
true_positives = true_positives.size()[0]
ts_accuracy = true_positives/test.size()[0]
print('ts_accuracy '+str(ts_accuracy)+'\n')

fms took 0.2037050724029541
ts_accuracy 0.0



In [43]:
print(consequences.size())

torch.Size([1462, 545])
